<h1 align="center"><b><span style="color: firebrick">"Crawling" Financial Statement data from cafef.vn by SELENIUM lib</span></b></h1>

<p>
    <h3>Goal: </h3>
    Stockcode: <b>Any stockcode</b> <br>
    Year: <b>2019-2023</b><br>
    Results: <b>3 types of FS -> 3 file excel: Stockcode_BS.xlsx, Stockcode_IS.xlsx, Stockcode_CF.xlsx</b>
</p>

## <span style="color: blue">1. Essential libraries:</span>

**pip install selenium**

**pip install webdriver_manager**

**pip install pandas**

**pip install xlsxwriter**

In [1]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
import pandas as pd

import os

## <span style="color: blue">2. Load web driver</span>

In [ ]:
# Chrome
service = ChromeService(ChromeDriverManager().install())
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless=new")
driver_auto = webdriver.Chrome(service=service, options=chrome_options)
print(driver_auto.service.path)
# driver_auto.get("https://vnexpress.net")

## <span style="color: blue">3. Crawling data</span>

https://s.cafef.vn/bao-cao-tai-chinh/FPT/BSheet/2023/0/0/0/bao-cao-tai-chinh-cong-ty.chn

https://s.cafef.vn/bao-cao-tai-chinh/FPT/IncSta/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn

https://s.cafef.vn/bao-cao-tai-chinh/FPT/CashFlow/2023/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn

In [3]:
def get_elements_value(items, convert_numeric=True):
    _ls = []
    if convert_numeric == False:
        for x in items:
            _ls.append(x.text)
    else:
        for x in items:
            num = x.text.strip().replace(',', '')
            if num.isnumeric():
                _ls.append(int(num))
            else:
                _ls.append(num)
    return _ls

def year_col_process(driver, col):
    x_path = f"//*[@id='tableContent']/tbody/tr/td[{col}]"
    rows = driver.find_elements(By.XPATH, x_path)
    return rows

def get_report_url(ticker, year, report_type):
    if report_type.upper() == 'BS':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/BSheet/{year}/0/0/0/bao-cao-tai-chinh-cong-ty.chn"
    if report_type.upper() == 'IS':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/IncSta/{year}/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn"
    if report_type.upper() == 'CF':
        return f"https://s.cafef.vn/bao-cao-tai-chinh/{ticker}/CashFlow/{year}/0/0/0/ket-qua-hoat-dong-kinh-doanh-cong-ty.chn"
    
    return ""
    
def get_excel_data(driver, ticker, from_year, to_year, report_type, save_path=None):
    year = to_year    
    data = {}
    criteria_names = {}
    runable = True
    while(runable):
        url = get_report_url(ticker, year, report_type)
        print(url)
        driver.get(url)
        driver.implicitly_wait(5)

        if 'criteria' not in criteria_names:
            name_elements = year_col_process(driver, 1)
            criteria_names['criteria'] = get_elements_value(name_elements, False)

        year_elements = driver.find_elements(By.XPATH, "//*[@id='tblGridData']/tbody/tr/td")
        index_cols = {}
        i = 1
        for item in year_elements:
            str_item = item.text.strip()
            if str_item.isnumeric():
                index_cols[str_item] = i
            i += 1

        index_cols = dict(sorted(index_cols.items(), reverse=True))

        y = 0
        for key in index_cols:
            col = index_cols[key]
            items = year_col_process(driver, col)
            data[key] = get_elements_value(items)
            print(key)

            y = int(key)
            if y == from_year:
                runable = False
                break

        if runable == True:
            year = y - 1

    data = dict(sorted(data.items()))  # Sort dict
    data = criteria_names | data       # Merge two dicts into one
    df = pd.DataFrame(data)

    if save_path is None:
        save_as = f"{ticker}_{report_type}.xlsx"
    else:
        save_as = f"{save_path}/{ticker}_{report_type}.xlsx"

    writer = pd.ExcelWriter(save_as,
                           engine='xlsxwriter',
                           engine_kwargs={'options': {'strings_to_numbers': True}})
    df.to_excel(writer, sheet_name=report_type, index=False)
    writer.close()

    return 1

In [ ]:
#customize ticker to get data
TICKERS = ['...']
TO_YEAR = 2023
LOOK_BACK = 5
FROM_YEAR = TO_YEAR - LOOK_BACK + 1
report_types = ['BS', 'IS', 'CF']
for TICKER in TICKERS:
    for report in report_types:
        if report == 'BS':
            save_path = 'Data/BS'
            get_excel_data(driver_auto, TICKER, FROM_YEAR, TO_YEAR, report, save_path)
        elif report == 'IS':
            save_path = 'Data/IS'
            get_excel_data(driver_auto, TICKER, FROM_YEAR, TO_YEAR, report, save_path)
        if report == 'CF':
            save_path = 'Data/CF'
            get_excel_data(driver_auto, TICKER, FROM_YEAR, TO_YEAR, report, save_path)

driver_auto.quit()
os.startfile(os.getcwd())